In [1]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict

In [2]:

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')

In [3]:
# Load the netCDF file
file      = 'CMEMS_cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_April-May2024.nc'
filepath  = os.path.join(satellite_dir, file)

# Load the netCDF file
ds = xr.open_dataset(filepath)

# Extract the relevant data
depth = ds['depth']
uo = ds['uo']
vo = ds['vo']

# Calculate depth intervals
depth_diff = np.diff(depth, append=depth[-1])

# Convert depth_diff to DataArray
depth_diff_da = xr.DataArray(depth_diff, coords={'depth': depth}, dims='depth')

# Compute depth-weighted averages for each day
def weighted_avg(da, weights):
    weighted_sum = (da * weights).sum(dim='depth')
    total_weights = weights.sum(dim='depth')
    return weighted_sum / total_weights

# Apply the weighted average function for each day
weighted_uo = uo.groupby('time').map(weighted_avg, args=(depth_diff_da,))
weighted_vo = vo.groupby('time').map(weighted_avg, args=(depth_diff_da,))

# Create a new dataset with the averaged data
averaged_ds = xr.Dataset({
    'uo': weighted_uo,
    'vo': weighted_vo
})

# Add attributes
averaged_ds['uo'].attrs = uo.attrs
averaged_ds['vo'].attrs = vo.attrs
averaged_ds.attrs = ds.attrs

# Save the averaged data to a new netCDF file
output_file      = 'CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_averaged.nc'
output_file_path = os.path.join(satellite_dir, output_file)
averaged_ds.to_netcdf(output_file_path)

C:\Users\hanshil\AppData\Local\Temp\ipykernel_13716\2922987076.py:26: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  weighted_uo = uo.groupby('time').map(weighted_avg, args=(depth_diff_da,))
C:\Users\hanshil\AppData\Local\Temp\ipykernel_13716\2922987076.py:27: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  weighted_vo = vo.groupby('time').map(weighted_avg, args=(depth_diff_da,))


In [4]:
# Load the netCDF file
file = 'CMEMS_cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_April-May2024.nc'
filepath = os.path.join(satellite_dir, file)

# Load the netCDF file
ds = xr.open_dataset(filepath)

# Extract the relevant data
depth = ds['depth']
uo = ds['uo']
vo = ds['vo']

# Determine the index of the end depth bin (1000m)
end_depth_value = 1000  # Assuming 1000m is the end depth bin you are interested in
end_depth_index = (depth == end_depth_value).argmax().item()

# Extract uo and vo at the end depth bin
uo_1000m = uo.isel(depth=end_depth_index)
vo_1000m = vo.isel(depth=end_depth_index)

# Create a new dataset with the extracted data
ds_1000m = xr.Dataset({
    'uo': uo_1000m,
    'vo': vo_1000m
})

# Add attributes
ds_1000m['uo'].attrs = uo.attrs
ds_1000m['vo'].attrs = vo.attrs
ds_1000m.attrs = ds.attrs

# Save the dataset to a new NetCDF file
output_file_1000m = 'CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_1000m.nc'
output_file_path_1000m = os.path.join(satellite_dir, output_file_1000m)
ds_1000m.to_netcdf(output_file_path_1000m)

print(f"File saved at {output_file_path_1000m}")


PermissionError: [Errno 13] Permission denied: 'c:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\satellite\\CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_1000m.nc'